In [7]:
import pyiomica as pio
from pyiomica import dataStorage as ds
from pyiomica import visualizationFunctions as vf
from pyiomica import categorizationFunctions as cf
from pyiomica import visibilityGraphCommunityDetection as cd
from pyiomica.extendedDataFrame import DataFrame

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.colors as colors
import numpy as np
import pandas as pd

import community as cy
from networkx.algorithms import community
import itertools
###no warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
### function to get the median/mean of dataframe, ignore missing data
def __get_m(dataset, type='median'):
    '''
    input:
        dataset: pandas data frame, including "Missing"
        type: string, "median" or "mean",default = "median"
    output:
        the median or mean of dataset, exclude "Missing"
    '''
    
    size = dataset.shape
    dataset_median = np.zeros(size[1])

    for i in range (0, size[1]):
        temp = dataset.iloc[:,i]
        temp = temp.dropna()
        a = temp.astype('float').values
        if type == "mean":
            dataset_median[i] = np.mean(a)
        else:
            dataset_median[i] = np.median(a)
            
    return dataset_median

In [9]:
#function to plot the community as color bar
def __plotCommunityAsHeatmap(data, times, fileName, title='', figsize=(8,4), ylabel='Signal Intensity',
                             cmap='jet', graph_type='natural',weight=None, withsign=False, direction=None, cutoff=None):
    '''plot time series and community structure as heatmap, nodes in same community with same color
    
    Args:
        data: Numpy 2-D array of floats
        
        times: Numpy 1-D array of floats
        
        fileName: name of the figure file to save
        
        title: the figure title,default is empty
        
        figsize: tuple of int, Default (8,4)        
            Figure size in inches
        
        cmap: the color map to plot heatmap
        
        graph_type: string, default: 'natural'
            "horizontal", Horizontal Visibility Graph
            "natural",natural Visibility Graph
            "dual_horizontal", dual perspective horizontal visibility graph
            "dual_natural", dual perspective natural visibility graph
            
        weight: str, default:None
            None: no weighted
            'time': weight = abs(times[i] - times[j])
            'tan': weight = abs((data[i] - data[j])/(times[i] - times[j])) + 10**(-8)
            'distance': weight = A[i, j] = A[j, i] = ((data[i] - data[j])**2 + (times[i] - times[j])**2)**0.5
            
        withsign: boolean, Default False
            Whether to return the sign of adjacency matrix, 
            If True, the link from Natural perspective VG is positive, the link from reflected perspective VG is negative 
            Else, the are all positive
            
        direction:str, default is None, the direction that nodes aggregate to communities
            None: no specfic direction, e.g. both sieds
            left: nodes can only aggregate to the lefe side hubs, e.g. early hubs
            right: nodes can only aggregate to the right side hubs, e.g. later hubs
        
        cutoff: will be used to combine initial communities, e.g. whenever the shortest path length of 
            two adjacent hub nodes is smaller than cutoff, the communities with the two hub nodes will be combined.
            the cutoff can be int,float or string
            int or float: the percentile of all shortest path length distribution, between 0 ~ 100
            'auto': use optimized cutoff
            None: no cutoff
            the default is None
        
    Returns:
        None
        
    Usage:
        __plotCommunityAsHeatmap(data, times, 'Test.png', 'Test Data')
    '''
    methods = ['GN', 'LN','PL']
    G_nx, A = cd.createVisibilityGraph(data, times, graph_type=graph_type, weight=weight, withsign=withsign)
        
    community_pl = cd.communityDetectByPathLength(G_nx, direction=direction, cutoff = cutoff) 
    heatMapData = []
   
    temp1 = np.zeros(len(times))
    temp2 = np.zeros(len(times))
    temp3 = np.zeros(len(times)) 


    comp = community.girvan_newman(G_nx)
    
    k = len(community_pl)  
            
    limited = itertools.takewhile(lambda c: len(c) <= k, comp)
    for communities in limited:
        community_gn = (list(sorted(c) for c in communities))
    for i, row in enumerate(community_gn):
        for j in row:
            temp3[j] = i
    heatMapData.append(temp3)
    
    community_ln = cy.best_partition(G_nx)
    temp2 = list(community_ln.values())    
    heatMapData.append(temp2)

    for i, row in enumerate(community_pl):
        for j in row:
            temp1[j] = i
    heatMapData.append(temp1)

    Ttimes = np.array(times)
    fig = plt.figure(figsize=figsize)
    ax1 = plt.subplot(211)
    ax1.bar(np.arange(0.25,len(times)+0.25,1), data, width=0.5)
    ax1.axhline(y=0, color='k')
    ax1.set_ylabel(ylabel, fontsize=16)
    ax1.set_xticks(np.arange(0,len(data),1))
    ax1.set_xticklabels(Ttimes[np.arange(0,len(times),1)],fontsize=16)
    ax1.set_yticks([])
    ax1.set_xlim(left=0,right=len(data))
    ax1.set_ylim(1.2*min(data),1.2*max(data))
    ax1.set_title(title,fontsize=20)
    

    ax2 = plt.subplot(212, sharex=ax1)
    #im2 = ax2.imshow(heatMapData)
    im2 = ax2.pcolor(heatMapData, cmap=cmap)
    ax2.set_xticks(np.arange(0,len(data),1))
    ax2.set_xticklabels(Ttimes[np.arange(0,len(times),1)],fontsize=16)
    ax2.set_yticks(np.arange(0,len(heatMapData),1), minor=True) 
    ax2.set_yticks(np.arange(0.5,len(heatMapData),1)) 
    ax2.set_yticklabels(methods,fontsize=16)
    ax2.grid(which="minor", color="w", axis='y', linestyle='-', linewidth=3)
    ax2.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
    for edge, spine in ax2.spines.items():
        spine.set_visible(False)
        
    fig.tight_layout()
    fig.savefig(fileName, dpi=600)
    plt.close(fig)
    
    
    return None

In [10]:
###The example below is our saliva hourly data

### Categorize the saliva omics data

dataName1 = 'SLV_Hourly1TimeSeries'
df_data_processed_H1 = ds.read(dataName1+'_df_data_transformed', hdf5fileName=pio.os.path.join('data',dataName1+'.h5'))
dataName2 = 'SLV_Hourly2TimeSeries'
df_data_processed_H2 = ds.read(dataName2+'_df_data_transformed', hdf5fileName=pio.os.path.join('data',dataName2+'.h5'))

dataName = 'SLV_Delta'
saveDir = pio.os.path.join('data', '')
df_data = ds.read(dataName+'_df_data_transformed', hdf5fileName=pio.os.path.join('data',dataName+'.h5'))
cf.calculateTimeSeriesCategorization(df_data, dataName, saveDir,  p_cutoff=0.05,NumberOfRandomSamples = 10**5)
cf.clusterTimeSeriesCategorization(dataName, saveDir)
#cf.visualizeTimeSeriesCategorization(dataName, saveDir)

###Get the data of a clustering object of Lag1
ExampleClusteringObject = ds.read(pio.os.path.join(saveDir, 'consolidatedGroupsSubgroups', 'SLV_Delta_LAG1_Autocorrelations_GroupsSubgroups'))

# Use Group 1, Subgroup 2 data
df_data = ExampleClusteringObject[1][2]['data']

# Read intensities of all signals and the measurement time points (here positions)
intensities, positions = df_data.values, df_data.columns.values

#Get the average of data
df_data = pd.DataFrame(intensities)
m_data  = __get_m(df_data, type = "mean")
tp = list(range(len(m_data)))

# Create dual-prespective visibility graph
g_nx, A_nx = cd.createVisibilityGraph(m_data,tp,"dual_natural", weight = 'distance')

# Detect community
communities = cd.communityDetectByPathLength(g_nx, cutoff='auto')
com = (communities, g_nx)
# Plot graph with community
vf.makeVisibilityGraph(m_data, tp, 'draft_fig/fig5', 'A', layout='line',communities=com, 
                       level=0.8,figsize = (10,10), extension='.eps')
#plot community structure compare with other methods
__plotCommunityAsHeatmap(m_data, tp, './draft_fig/fig5/A2.eps', title = 'saliva delta Lag1 G1S2 ' ,figsize = (10,5),cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')



 ---------------------------------------------------------------------- 
	Processing SLV_Delta (Autocorrelations) 
 ----------------------------------------------------------------------
Filtering out all-zero signals
Removed  0 signals out of 5823.
Remaining  5823 signals!
Filtering out first time point zeros signals
Filtering out low-quality signals (with more than 25.0% zero points)
Tagging 0.0 values with nan
Tagging low values (<=1.0) with 1.0

Removing constant signals. Cutoff value is 0.0
Removed  0 signals out of 5823.
Remaining  5823 signals!
Quantiles: [1.0, 0.0268710939703153, 0.5849163710105024, 0.2218884091243272, 0.4266986136108797, 0.2936560413043456, 0.3659137391564724, 0.3213790779740068, 0.3498494471868286, 0.2481654181599319, 0.2916285225209877, 0.2254708250715485] 

Calculating spikes cutoffs...
Filtering out low-quality signals (with more than 25.0% zero points)
Normalizing signals to unity

Removing constant signals. Cutoff value is 0.0
Removed  0 signals out of 

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 6, 16, 22, 23]
current cutoff is auto, the optimized percentiles cutoff is 0.000098 


In [11]:
#example below is Stanford wearables radiation data

#Import Radiation data
data = pd.read_excel('./data/Stanford_Wearables_Radiation_06072016.xlsx',header=[0,1])
data.columns = ['DateTime','Hp'] 

#Split DateTime to get Date
new = data["DateTime"].str.split(" ", n = 1, expand = True) 
data["Date"]= new[0] 

#extract the continous meatures time range, start from 2015-12-17 11
ix = data.index[data['DateTime'] == '2015-12-17 11'].tolist()
df = data[ix[0]:]

#change the unit
idx = df.index[df['Hp'] > 100]
df.loc[idx[0]:idx[-1],'Hp'] *=0.001

#get hourly radiation
diff = df['Hp'] - df['Hp'].shift()
df['Hourlydiff'] = diff
df.index = np.arange(0, len(df))
#drop first row
df.dropna(axis=0, subset=('Hourlydiff', ),inplace=True)
          
#Get the first flight date 2015-12-23'
ixs = df.index[df['DateTime'] == '2015-12-22 23'].tolist()
ixe = df.index[df['DateTime'] == '2015-12-23 23'].tolist()
df1 = df[ixs[0]:ixe[0]]

#reset index
df1.index = np.arange(0, len(df1))

ftimes1 = df1.index.values
fdata1 = df1['Hourlydiff'].values

#plot communities
__plotCommunityAsHeatmap(fdata1, ftimes1, './draft_fig/fig5/C1.eps',figsize = (10,5),
                         title = '12-23-2015' ,ylabel='mRem',cmap='jet',graph_type='natural', weight='distance',direction='left',cutoff='auto')


#get the second flight date '2016-01-02'

ixs = df.index[df['DateTime'] == '2016-01-01 23'].tolist()
ixe = df.index[df['DateTime'] == '2016-01-02 23'].tolist()
df2 = df[ixs[0]:ixe[0]]

#reset index
df2.index = np.arange(0, len(df2))

ftimes2 = df2.index.values
fdata2 = df2['Hourlydiff'].values

#plot communities 
__plotCommunityAsHeatmap(fdata2, ftimes2, './draft_fig/fig5/C2.eps',figsize = (10,5),
                         title = '01-02-2016' ,ylabel='mRem',cmap='jet',graph_type='natural', weight='distance',direction='left',cutoff='auto')

#get the third flight date ''2016-01-05''

ixs = df.index[df['DateTime'] == '2016-01-04 23'].tolist()
ixe = df.index[df['DateTime'] == '2016-01-05 23'].tolist()
df3 = df[ixs[0]:ixe[0]]

#reset index
df3.index = np.arange(0, len(df3))

ftimes3 = df3.index.values
fdata3 = df3['Hourlydiff'].values

#plot communities
__plotCommunityAsHeatmap(fdata3, ftimes3, './draft_fig/fig5/C3.eps',figsize = (10,5),
                         title = '01-05-2016' ,ylabel='mRem',cmap='jet',graph_type='natural', weight='distance',direction='left',cutoff='auto')

#get the forth flight date '2016-01-10'

ixs = df.index[df['DateTime'] == '2016-01-09 23'].tolist()
ixe = df.index[df['DateTime'] == '2016-01-10 23'].tolist()
df4 = df[ixs[0]:ixe[0]]

#reset index
df4.index = np.arange(0, len(df4))

ftimes4 = df4.index.values
fdata4 = df4['Hourlydiff'].values

#plot communities
__plotCommunityAsHeatmap(fdata4, ftimes4, './draft_fig/fig5/C4.eps',figsize = (10,5),
                         title = '01-10-2016' ,ylabel='mRem',cmap='jet',graph_type='natural', weight='distance',direction='left',cutoff='auto')

graph type is: natural
weight is: distance
direction type is: left
the shortest path is: [0, 7, 12, 23]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: natural
weight is: distance
direction type is: left
the shortest path is: [0, 11, 19, 20, 23]
current cutoff is auto, the optimized percentiles cutoff is 9.091690 
graph type is: natural
weight is: distance
direction type is: left
the shortest path is: [0, 17, 23]
current cutoff is auto, the optimized percentiles cutoff is 22.727492 
graph type is: natural
weight is: distance
direction type is: left
the shortest path is: [0, 13, 23]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 


In [12]:
#example below is Stanford iPOP data

#import data
data = pd.read_excel('./data/Stanford_iPOP_sample.xlsx',header=0, index_col=0)
length = len(data.index.values)
#get cytokines names list
names = list(data)
names = names[1:-1]
names.reverse()

#construct the communities matrix & standardized signal intensity data frame

norm_data = pd.DataFrame()
Z = []
Zln = []
Zgn = []

# get the communities of each type of signal
for i, n in enumerate (names):
    df = data[n]
    df.dropna(inplace=True)
    times = df.index.values

    val = np.array(df)

    G_nx, A = cd.createVisibilityGraph(val, times, graph_type='natural', weight='distance')
    community_pl = cd.communityDetectByPathLength(G_nx, cutoff = 'auto')
    
    community_ln = cy.best_partition(G_nx)
    
    comp = community.girvan_newman(G_nx)
    k = len(community_pl)    
    limited = itertools.takewhile(lambda c: len(c) <= k, comp)
    for communities in limited:
        community_gn = (list(sorted(c) for c in communities))

    timepoint=nx.get_node_attributes(G_nx,'timepoint')
    temp = np.array([np.nan]*length)
    for i, row in enumerate(community_pl):
        for j in row:
            temp[int(timepoint[j])-1] = i
    Z.append(temp)
    
    temp2 = np.array([np.nan]*length)
    for key, value in community_ln.items():
        temp2[int(timepoint[key])-1] = value
    Zln.append(temp2)
    
    temp3 = np.array([np.nan]*length)
    for i, row in enumerate(community_gn):
        for j in row:
            temp3[int(timepoint[j])-1] = i        
    Zgn.append(temp3)
    
    #get standardized signal 
    x = data[n]
    x_nona = x.dropna()
    t=(x - x_nona.mean())/x_nona.std()
    norm_data[n] = t
    
Zm = np.array(Z)
heatmapdata = np.ma.masked_where(np.isnan(Zm),Zm)

Zmln = np.array(Zln)
heatmapdataln = np.ma.masked_where(np.isnan(Zmln),Zmln)

Zmgn = np.array(Zgn)
heatmapdatagn = np.ma.masked_where(np.isnan(Zmgn),Zmgn)

#plot communities PL
xlabel = ['H','H','S','H','H','H','Ax','Ax','Ax','Ax','H','Im','H','H']
fig = plt.figure(figsize=(10,10)) 
ax1 = fig.add_subplot()
im = ax1.pcolor(heatmapdata,norm=colors.PowerNorm(gamma=0.5), cmap='jet',
               edgecolors='white', linewidths=1)
ax1.set_title('PL method', fontdict={'color': 'k'},fontsize=30)
ax1.set_yticklabels(names,fontsize=24)
ax1.set_yticks(np.arange(0+0.5,len(names)+0.5,1))
ax1.set_xticks(np.arange(0+0.5,len(xlabel)+0.5,1))
ax1.set_xticklabels(xlabel,fontsize=24)
ax1.set_xlim(left=0,right=len(xlabel))
ax1.set_yticks(np.arange(0,len(names),1), minor = True)
ax1.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
ax1.grid(b=None, which='minor', axis='y', color='white',lw=5) 
ax1.yaxis.tick_right()
for edge, spine in ax1.spines.items():
    spine.set_visible(False)
fig.tight_layout()
plt.savefig('./draft_fig/fig5/B1.eps',dpi=600)


#plot communities LN
xlabel = ['H','H','S','H','H','H','Ax','Ax','Ax','Ax','H','Im','H','H']
fig = plt.figure(figsize=(10,10)) 
ax1 = fig.add_subplot()
im = ax1.pcolor(heatmapdataln,norm=colors.PowerNorm(gamma=0.5), cmap='jet',
               edgecolors='white', linewidths=1)
ax1.set_title('LN method', fontdict={'color': 'k'},fontsize=30)
ax1.set_yticklabels(names,fontsize=24)
ax1.set_yticks(np.arange(0+0.5,len(names)+0.5,1))
ax1.set_xticks(np.arange(0+0.5,len(xlabel)+0.5,1))
ax1.set_xticklabels(xlabel,fontsize=24)
ax1.set_xlim(left=0,right=len(xlabel))
ax1.set_yticks(np.arange(0,len(names),1), minor = True)
ax1.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
ax1.grid(b=None, which='minor', axis='y', color='white',lw=5)   
#ax1.yaxis.tick_right()
for edge, spine in ax1.spines.items():
    spine.set_visible(False)
fig.tight_layout()
plt.savefig('./draft_fig/fig5/B2.eps',dpi=600)

#plot communities GN
xlabel = ['H','H','S','H','H','H','Ax','Ax','Ax','Ax','H','Im','H','H']
fig = plt.figure(figsize=(10,10)) 
ax1 = fig.add_subplot()
im = ax1.pcolor(heatmapdatagn,norm=colors.PowerNorm(gamma=0.5), cmap='jet',
               edgecolors='white', linewidths=1)
ax1.set_title('GN method', fontdict={'color': 'k'},fontsize=30)
ax1.set_yticklabels(names,fontsize=24)
ax1.set_yticks(np.arange(0+0.5,len(names)+0.5,1))
ax1.set_xticks(np.arange(0+0.5,len(xlabel)+0.5,1))
ax1.set_xticklabels(xlabel,fontsize=24)
ax1.set_xlim(left=0,right=len(xlabel))
ax1.set_yticks(np.arange(0,len(names),1), minor = True)
ax1.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
ax1.grid(b=None, which='minor', axis='y', color='white',lw=5)
ax1.yaxis.tick_right()
for edge, spine in ax1.spines.items():
    spine.set_visible(False)
fig.tight_layout()
plt.savefig('./draft_fig/fig5/B3.eps',dpi=600)

#plot the signals
masked_data = np.ma.masked_where(np.isnan(norm_data.T),norm_data.T)

xlabel = ['H','H','S','H','H','H','Ax','Ax','Ax','Ax','H','Im','H','H']
fig = plt.figure(figsize=(10,10)) 
ax2= fig.add_subplot()
im = ax2.pcolor(masked_data,norm=colors.PowerNorm(gamma=0.5), cmap='jet',
               edgecolors='white', linewidths=1)
ax2.set_title('Signal Intensity', fontdict={'color': 'k'},fontsize=30)
ax2.set_yticklabels(names,fontsize=24)
ax2.set_yticks(np.arange(0+0.5,len(names)+0.5,1))
ax2.set_xticks(np.arange(0+0.5,len(xlabel)+0.5,1))
ax2.set_xticklabels(xlabel,fontsize=24)
#ax2.yaxis.tick_right()

ax2.set_xlim(left=0,right=len(xlabel))
ax2.set_yticks(np.arange(0,len(names),1), minor = True)
ax2.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
ax2.grid(b=None, which='minor', axis='y', color='white',lw=5)    
for edge, spine in ax1.spines.items():
    spine.set_visible(False)
fig.tight_layout()    
plt.savefig('./draft_fig/fig5/B4.eps',dpi=600)

graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 7, 8, 9, 11, 13]
current cutoff is auto, the optimized percentiles cutoff is 26.038251 
graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 6, 9, 13]
current cutoff is auto, the optimized percentiles cutoff is 23.851790 
graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 8, 13]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 8, 13]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 8, 13]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: natural
weight is: distance
direction type is: None
the shortest path is: [0, 8, 13]
current cutoff is auto, the optimiz